In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import json
import numpy as np
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm

device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

In [2]:
#First, we load the respective CLIP model
model = SentenceTransformer('clip-ViT-B-32', device=device)

In [3]:
# Rule load = document = corpus
data = 'data/aihub_rules_prev.json'
with open(data, 'r', encoding="UTF-8") as j:
    aihub_rule = json.load(j)

normal_rule = aihub_rule['normal']
abnormal_rule = aihub_rule['abnormal']
combined_rules = normal_rule + abnormal_rule

In [4]:
def find_rank_of_answer_in_results(final_indices, answer_index):
    try:
        return final_indices.index(answer_index) + 1
    except ValueError:
        return -1

In [5]:
root_path = '/data1/sliver/jwsuh/construction_dataset/aihub/llava/llava_image_result_with_obj'
files = sorted([i for i in os.listdir(root_path) if i.endswith('.jpg')])

In [7]:
text_emb = model.encode(combined_rules)

In [19]:
save_right_index = {}
save_wrong_index = {}
final_top_k_indices_list = []

for final_top_k in [50]:
    correct = 0
    save_right_index[final_top_k] = {}
    save_wrong_index[final_top_k] = {}
    
    for idx, file in enumerate(tqdm(files)):
        img_emb = model.encode(Image.open(os.path.join(root_path, file)))
        cos_scores = util.cos_sim(img_emb, text_emb)[0]
        top_results =  torch.topk(cos_scores, k=50)
        final_top_k_indices = top_results[1].tolist()
        # final_top_k_indices = biencoder_initial_search(caption['outputs'], idx, top_k=50)
        final_top_k_indices_list.append(final_top_k_indices)
        
        answer = file.split('_')[2]
        if answer[0] == 'Y':
            answer_index = int(answer[2:]) - 1
        elif answer[0] == 'N':
            answer_index = int(answer[2:]) + 49
            
        rank = find_rank_of_answer_in_results(final_top_k_indices, answer_index)

        if rank <= final_top_k and rank != -1:
            correct += 1
            save_right_index[final_top_k][idx] = final_top_k_indices
        else:
            save_wrong_index[final_top_k][idx] = final_top_k_indices

    print(f"Top-{final_top_k} accuracy:", correct / len(files))

  0%|          | 0/957 [00:00<?, ?it/s]

Top-50 accuracy: 0.910135841170324


In [20]:
for final_top_k in [1, 5, 10, 15, 20, 25, 30]:
    correct = 0
    
    for idx, file in enumerate(tqdm(files)):
        
        final_top_k_indices = final_top_k_indices_list[idx]

        answer = file.split('_')[2]
        if answer[0] == 'Y':
            answer_index = int(answer[2:]) - 1
        elif answer[0] == 'N':
            answer_index = int(answer[2:]) + 49
            
        rank = find_rank_of_answer_in_results(final_top_k_indices, answer_index)

        if rank <= final_top_k and rank != -1:
            correct += 1

    print(f"Top-{final_top_k} accuracy:", correct / len(files))

  0%|          | 0/957 [00:00<?, ?it/s]

Top-1 accuracy: 0.08881922675026123


  0%|          | 0/957 [00:00<?, ?it/s]

Top-5 accuracy: 0.29571577847439917


  0%|          | 0/957 [00:00<?, ?it/s]

...

Top-20 accuracy: 0.6781609195402298


  0%|          | 0/957 [00:00<?, ?it/s]

Top-25 accuracy: 0.741901776384535


  0%|          | 0/957 [00:00<?, ?it/s]

Top-30 accuracy: 0.786833855799373
